In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords
import nltk.tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')

import re


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package punkt to C:\Users\Success
[nltk_data]     Corporate\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Success
[nltk_data]     Corporate\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Success
[nltk_data]     Corporate\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to C:\Users\Success
[nltk_data]     Corporate\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Success
[nltk_data]     Corporate\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Success
[nltk_data]     Corpo

# Importing dataset

In [2]:
dataset = pd.read_csv("fake news class 300 edited.csv",encoding='ISO-8859-1') #read csv + encoding
dataset.head()

,id,title,text,label
0,0.0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1.0
1,1.0,NaN,Did they post their votes for Hillary already?,1.0
2,2.0,UNBELIEVABLE! OBAMAâS ATTORNEY GENERAL SAYS ...,"Now, most of the demonstrators gathered last ...",1.0
3,3.0,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0.0
4,4.0,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1.0


In [3]:
dataset.isnull().sum(axis=0)  
dataset.rename(columns={'Unnamed: 0':'id'}, inplace = True)

In [4]:
dataset['id'].replace('  ', np.nan, inplace=True)
dataset= dataset.dropna(subset=['id'])  

# Cleaning dataset


In [5]:
dataset.dropna(how='any', inplace=True) #dropped all rows with na values so set dtype as int instead of float
#we are dropping 2000 ish values cos its less than 5% of our entire dataset - refer to 5% of missing values rule

In [6]:
dataset.id= pd.factorize(dataset.id)[0]
dataset['title'] = dataset.title.astype(str)
dataset['text'] = dataset.text.astype(str)
dataset.label= pd.factorize(dataset.label)[0]

# Making new columns


In [7]:
dataset['title_char_count'] = dataset['title'].str.len().astype(int) #new column for no. of char
dataset['text_char_count'] = dataset['text'].str.len().astype(int)

dataset['title_char_count'] = dataset.title_char_count.astype(int) #changing all dtypes of new columns

dataset['title_word_count'] = dataset['title'].str.split().str.len().astype(int) #new column for no. of words
dataset['text_word_count'] = dataset['text'].str.split().str.len().astype(int)

dataset.head()


,id,title,text,label,title_char_count,text_char_count,title_word_count,text_word_count
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,0,130,5049,18,871
2,1,UNBELIEVABLE! OBAMAâS ATTORNEY GENERAL SAYS ...,"Now, most of the demonstrators gathered last ...",0,145,216,18,34
3,2,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,1,105,8120,16,1321
4,3,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",0,101,1932,16,329
5,4,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...,0,78,1530,13,244


# Information on dataset


In [8]:
dataset.shape
dataset.info()
dataset.dtypes

<class 'pandas.core.frame.DataFrame'>
Int64Index: 295 entries, 0 to 305
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                295 non-null    int64 
 1   title             295 non-null    object
 2   text              295 non-null    object
 3   label             295 non-null    int64 
 4   title_char_count  295 non-null    int32 
 5   text_char_count   295 non-null    int32 
 6   title_word_count  295 non-null    int32 
 7   text_word_count   295 non-null    int32 
dtypes: int32(4), int64(2), object(2)
memory usage: 16.1+ KB


id                   int64
title               object
text                object
label                int64
title_char_count     int32
text_char_count      int32
title_word_count     int32
text_word_count      int32
dtype: object

# Removal of Symbols


In [9]:
def remove_symbols(string):
    string = re.sub('[^a-zA-Z!?\']', ' ', string) #replacement
    return " ".join(string.split()) #eliminate multiple spaces

dataset['text'] = dataset['text'].apply(remove_symbols)
dataset['title'] = dataset['title'].apply(remove_symbols)
dataset.head()

,id,title,text,label,title_char_count,text_char_count,title_word_count,text_word_count
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,0,130,5049,18,871
2,1,UNBELIEVABLE! OBAMA S ATTORNEY GENERAL SAYS MO...,Now most of the demonstrators gathered last ni...,0,145,216,18,34
3,2,Bobby Jindal raised Hindu uses story of Christ...,A dozen politically active pastors came here f...,1,105,8120,16,1321
4,3,SATAN Russia unvelis an image of its terrifyin...,The RS Sarmat missile dubbed Satan will replac...,0,101,1932,16,329
5,4,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...,0,78,1530,13,244


# Labelling of fake and real news


In [10]:
label_translated = np.where(dataset['label']==0, 'fake', 'real')
dataset.insert(loc = 4 ,column = 'label_translated',value = label_translated)

dataset.head()

,id,title,text,label,label_translated,title_char_count,text_char_count,title_word_count,text_word_count
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,0,fake,130,5049,18,871
2,1,UNBELIEVABLE! OBAMA S ATTORNEY GENERAL SAYS MO...,Now most of the demonstrators gathered last ni...,0,fake,145,216,18,34
3,2,Bobby Jindal raised Hindu uses story of Christ...,A dozen politically active pastors came here f...,1,real,105,8120,16,1321
4,3,SATAN Russia unvelis an image of its terrifyin...,The RS Sarmat missile dubbed Satan will replac...,0,fake,101,1932,16,329
5,4,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...,0,fake,78,1530,13,244


# Removal of Stopwords

In [11]:
from nltk.corpus import stopwords

def add_stopword_count_column(dataset, stop_words, text_column_name, stopword_count_column_name):
    index = 0
    for text in dataset[text_column_name]:
        textlist = text.split()
        stopword_number = 0
        for word in textlist:
            if word.lower() in stop_words:
                stopword_number += 1
        dataset.at[index, stopword_count_column_name] = stopword_number
        index += 1
    dataset[stopword_count_column_name] = dataset[stopword_count_column_name].fillna(0).astype(int)
    return dataset


stop_words = stopwords.words('english')

# Remove stop words from title column
dataset['title'] = dataset['title'].apply(lambda x: ' '.join([word for word in str(x).split() if word not in (stop_words)]))

# Add stopword count column for title
dataset = add_stopword_count_column(dataset, stop_words, 'title', 'title_stopword_count')

# Remove stop words from text column
dataset['text'] = dataset['text'].apply(lambda x: ' '.join([word for word in str(x).split() if word not in (stop_words)]))

# Add stopword count column for text
dataset = add_stopword_count_column(dataset, stop_words, 'text', 'text_stopword_count')

dataset.head()


,id,title,text,label,label_translated,title_char_count,text_char_count,title_word_count,text_word_count,title_stopword_count,text_stopword_count
0,0.0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment expected Barack Obama Members FYF F...,0.0,fake,130.0,5049.0,18.0,871.0,6,48
2,1.0,UNBELIEVABLE! OBAMA S ATTORNEY GENERAL SAYS MO...,Now demonstrators gathered last night exercisi...,0.0,fake,145.0,216.0,18.0,34.0,0,46
3,2.0,Bobby Jindal raised Hindu uses story Christian...,A dozen politically active pastors came privat...,1.0,real,105.0,8120.0,16.0,1321.0,0,16
4,3.0,SATAN Russia unvelis image terrifying new SUPE...,The RS Sarmat missile dubbed Satan replace SS ...,0.0,fake,101.0,1932.0,16.0,329.0,1,10
5,4.0,About Time! Christian Group Sues Amazon SPLC D...,All say one time someone sued Southern Poverty...,0.0,fake,78.0,1530.0,13.0,244.0,4,7


In [12]:
dataset.dropna(how='any', inplace=True) #dropped all rows with na values so set dtype as int instead of float

dataset['title_stopword_count'] = dataset.title_stopword_count.astype(int)
dataset['text_stopword_count'] = dataset.text_stopword_count.astype(int)


### Lemmatisation

In [13]:
lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
    
# Apply lemmatization to text column
dataset['text_lemmatized'] = dataset['text'].apply(lambda x: " ".join([lemmatizer.lemmatize(word, get_wordnet_pos(pos_tag)) 
                                                        for word, pos_tag in nltk.pos_tag(x.split())]))
# Apply lemmatization to title column
dataset['title_lemmatized'] = dataset['title'].apply(lambda x: " ".join([lemmatizer.lemmatize(word, get_wordnet_pos(pos_tag)) 
                                                          for word, pos_tag in nltk.pos_tag(x.split())]))


In [14]:
dataset.head()

,id,title,text,label,label_translated,title_char_count,text_char_count,title_word_count,text_word_count,title_stopword_count,text_stopword_count,text_lemmatized,title_lemmatized
0,0.0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment expected Barack Obama Members FYF F...,0.0,fake,130.0,5049.0,18.0,871.0,6,48,No comment expect Barack Obama Members FYF Fuk...,LAW ENFORCEMENT ON HIGH ALERT Following Threat...
2,1.0,UNBELIEVABLE! OBAMA S ATTORNEY GENERAL SAYS MO...,Now demonstrators gathered last night exercisi...,0.0,fake,145.0,216.0,18.0,34.0,0,46,Now demonstrator gather last night exercise co...,UNBELIEVABLE! OBAMA S ATTORNEY GENERAL SAYS MO...
3,2.0,Bobby Jindal raised Hindu uses story Christian...,A dozen politically active pastors came privat...,1.0,real,105.0,8120.0,16.0,1321.0,0,16,A dozen politically active pastor come private...,Bobby Jindal raise Hindu use story Christian c...
4,3.0,SATAN Russia unvelis image terrifying new SUPE...,The RS Sarmat missile dubbed Satan replace SS ...,0.0,fake,101.0,1932.0,16.0,329.0,1,10,The RS Sarmat missile dub Satan replace SS Fli...,SATAN Russia unvelis image terrify new SUPERNU...
5,4.0,About Time! Christian Group Sues Amazon SPLC D...,All say one time someone sued Southern Poverty...,0.0,fake,78.0,1530.0,13.0,244.0,4,7,All say one time someone sue Southern Poverty ...,About Time! Christian Group Sues Amazon SPLC D...


# Sentiment Generation - polarity 


In [15]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as sia

sia = sia()

# Define the get_polarity() function
def get_polarity(text):
    
    text = text[:text.find(' ', 30)] if len(text) > 30 else text
    text = " ".join(text.split()[:30])

    polarity_scores = sia.polarity_scores(text)
    polarity, score = sorted(polarity_scores.items(), key=lambda x: x[1], reverse=True)[:1][0]
    if polarity == 'pos':
        polarity = 'positive'
    elif polarity == 'neg':
        polarity = 'negative'
    elif polarity == 'neu':
        polarity = 'neutral'
    else:
        polarity = 'compound';
    return polarity, score

# Apply the get_polarity() function to the title column
polarity_score = dataset['title'].apply(get_polarity)
dataset['title_polarity'] = polarity_score.str[0]
dataset['title_polarity_score'] = polarity_score.str[1]

# Apply the get_polarity() function to the text column
polarity_score = dataset['text'].apply(get_polarity)
dataset['text_polarity'] = polarity_score.str[0]
dataset['text_polarity_score'] = polarity_score.str[1]


dataset.head()


,id,title,text,label,label_translated,title_char_count,text_char_count,title_word_count,text_word_count,title_stopword_count,text_stopword_count,text_lemmatized,title_lemmatized,title_polarity,title_polarity_score,text_polarity,text_polarity_score
0,0.0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment expected Barack Obama Members FYF F...,0.0,fake,130.0,5049.0,18.0,871.0,6,48,No comment expect Barack Obama Members FYF Fuk...,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,neutral,0.630,neutral,0.645
2,1.0,UNBELIEVABLE! OBAMA S ATTORNEY GENERAL SAYS MO...,Now demonstrators gathered last night exercisi...,0.0,fake,145.0,216.0,18.0,34.0,0,46,Now demonstrator gather last night exercise co...,UNBELIEVABLE! OBAMA S ATTORNEY GENERAL SAYS MO...,positive,0.511,neutral,1.000
3,2.0,Bobby Jindal raised Hindu uses story Christian...,A dozen politically active pastors came privat...,1.0,real,105.0,8120.0,16.0,1321.0,0,16,A dozen politically active pastor come private...,Bobby Jindal raise Hindu use story Christian c...,neutral,1.000,neutral,0.526
4,3.0,SATAN Russia unvelis image terrifying new SUPE...,The RS Sarmat missile dubbed Satan replace SS ...,0.0,fake,101.0,1932.0,16.0,329.0,1,10,The RS Sarmat missile dub Satan replace SS Fli...,SATAN Russia unvelis image terrify new SUPERNU...,neutral,0.519,neutral,1.000
5,4.0,About Time! Christian Group Sues Amazon SPLC D...,All say one time someone sued Southern Poverty...,0.0,fake,78.0,1530.0,13.0,244.0,4,7,All say one time someone sue Southern Poverty ...,About Time! Christian Group Sues Amazon SPLC D...,neutral,1.000,neutral,1.000


# Emotion Generation

In [21]:
!pip install text2emotion
import text2emotion as t2e

def get_emotion(sentence):
    sentence = sentence[:sentence.find(' ', 30)] if len(sentence) > 30 else sentence
    sentence = " ".join(sentence.split()[:30])
  
    emotion_scores = t2e.get_emotion(sentence)
    if not emotion_scores:
        return "neutral"
    emotion, score = max(emotion_scores.items(), key=lambda x: x[1])
    if score == 0.0:
        return "neutral"
    else:
        return emotion

    
dataset['title_emotion'] = dataset['title'].apply(get_emotion)
dataset['text_emotion'] = dataset['text'].apply(get_emotion)

dataset.head()


,id,title,text,label,label_translated,title_char_count,text_char_count,title_word_count,text_word_count,title_stopword_count,text_stopword_count,text_lemmatized,title_lemmatized,title_polarity,title_polarity_score,text_polarity,text_polarity_score,title_emotion,text_emotion
id,,,,,,,,,,,,,,,,,,,
0,0.0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment expected Barack Obama Members FYF F...,0.0,fake,130.0,5049.0,18.0,871.0,6,48,No comment expect Barack Obama Members FYF Fuk...,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,neutral,0.630,neutral,0.645,Surprise,Happy
1,1.0,UNBELIEVABLE! OBAMA S ATTORNEY GENERAL SAYS MO...,Now demonstrators gathered last night exercisi...,0.0,fake,145.0,216.0,18.0,34.0,0,46,Now demonstrator gather last night exercise co...,UNBELIEVABLE! OBAMA S ATTORNEY GENERAL SAYS MO...,positive,0.511,neutral,1.000,neutral,neutral
2,2.0,Bobby Jindal raised Hindu uses story Christian...,A dozen politically active pastors came privat...,1.0,real,105.0,8120.0,16.0,1321.0,0,16,A dozen politically active pastor come private...,Bobby Jindal raise Hindu use story Christian c...,neutral,1.000,neutral,0.526,neutral,neutral
3,3.0,SATAN Russia unvelis image terrifying new SUPE...,The RS Sarmat missile dubbed Satan replace SS ...,0.0,fake,101.0,1932.0,16.0,329.0,1,10,The RS Sarmat missile dub Satan replace SS Fli...,SATAN Russia unvelis image terrify new SUPERNU...,neutral,0.519,neutral,1.000,neutral,neutral
4,4.0,About Time! Christian Group Sues Amazon SPLC D...,All say one time someone sued Southern Poverty...,0.0,fake,78.0,1530.0,13.0,244.0,4,7,All say one time someone sue Southern Poverty ...,About Time! Christian Group Sues Amazon SPLC D...,neutral,1.000,neutral,1.000,Surprise,neutral


# Deleting empty rows after cleaning

In [17]:
empty_string_counts = (dataset == '').sum()
for col, count in empty_string_counts.iteritems():
    print(f"{col}:\t\t\t{count}")

dataset.replace('', np.nan, inplace=True)

id:			0
title:			0
text:			2
label:			0
label_translated:			0
title_char_count:			0
text_char_count:			0
title_word_count:			0
text_word_count:			0
title_stopword_count:			0
text_stopword_count:			0
text_lemmatized:			2
title_lemmatized:			0
title_polarity:			0
title_polarity_score:			0
text_polarity:			0
text_polarity_score:			0
title_emotion:			0
text_emotion:			0


In [18]:
dataset.isnull().sum(axis=0)

id                      0
title                   0
text                    2
label                   0
label_translated        0
title_char_count        0
text_char_count         0
title_word_count        0
text_word_count         0
title_stopword_count    0
text_stopword_count     0
text_lemmatized         2
title_lemmatized        0
title_polarity          0
title_polarity_score    0
text_polarity           0
text_polarity_score     0
title_emotion           0
text_emotion            0
dtype: int64

In [19]:
dataset.dropna(inplace=True)
dataset.reset_index(inplace=True, drop=True)
dataset.index.name = 'id'


dataset.shape
dataset.head()

,id,title,text,label,label_translated,title_char_count,text_char_count,title_word_count,text_word_count,title_stopword_count,text_stopword_count,text_lemmatized,title_lemmatized,title_polarity,title_polarity_score,text_polarity,text_polarity_score,title_emotion,text_emotion
id,,,,,,,,,,,,,,,,,,,
0,0.0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment expected Barack Obama Members FYF F...,0.0,fake,130.0,5049.0,18.0,871.0,6,48,No comment expect Barack Obama Members FYF Fuk...,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,neutral,0.630,neutral,0.645,Surprise,Happy
1,1.0,UNBELIEVABLE! OBAMA S ATTORNEY GENERAL SAYS MO...,Now demonstrators gathered last night exercisi...,0.0,fake,145.0,216.0,18.0,34.0,0,46,Now demonstrator gather last night exercise co...,UNBELIEVABLE! OBAMA S ATTORNEY GENERAL SAYS MO...,positive,0.511,neutral,1.000,neutral,neutral
2,2.0,Bobby Jindal raised Hindu uses story Christian...,A dozen politically active pastors came privat...,1.0,real,105.0,8120.0,16.0,1321.0,0,16,A dozen politically active pastor come private...,Bobby Jindal raise Hindu use story Christian c...,neutral,1.000,neutral,0.526,neutral,neutral
3,3.0,SATAN Russia unvelis image terrifying new SUPE...,The RS Sarmat missile dubbed Satan replace SS ...,0.0,fake,101.0,1932.0,16.0,329.0,1,10,The RS Sarmat missile dub Satan replace SS Fli...,SATAN Russia unvelis image terrify new SUPERNU...,neutral,0.519,neutral,1.000,neutral,neutral
4,4.0,About Time! Christian Group Sues Amazon SPLC D...,All say one time someone sued Southern Poverty...,0.0,fake,78.0,1530.0,13.0,244.0,4,7,All say one time someone sue Southern Poverty ...,About Time! Christian Group Sues Amazon SPLC D...,neutral,1.000,neutral,1.000,Surprise,neutral


In [20]:
# Save file
news_data.to_csv("cleaned_news_data.csv", sep=',', encoding='utf-8')

NameError: name 'news_data' is not defined